# creating nonRedundant UKB fasta for MS library search

8.8.19

mfpfox

## input
- ("countsfromfastaCCDS_filter_counts_all_AA.csv")

## output
- UKBccds_NONredundantSequence_18400.csv
- UKBccds_NONredundantSequence_18400.fasta


### fasta format
```bash
>sp|O95139|NDUB6_HUMAN NADH dehydrogenase [ubiquinone] 1 beta subcomplex subunit 6 OS=Homo sapiens OX=9606 GN=NDUFB6 PE=1 SV=3
MTGYTPDEKLRLQQLRELRRRWLKDQELSPREPVLPPQKMGPMEKFWNKFLENKSPWRKM
VHGVYKKSIFVFTHVLVPVWIIHYYMKYHVSEKPYGIVEKKSRIFPGDTILETGEVIPPM
KEFPDQHH

# version i am making will have below format
>sp|O95139|O95789 
MTGYTPDEKLRLQQLRELRRRWLKDQELSPREPVLPPQKMGPMEKFWNKFLENKSPWRKM
VHGVYKKSIFVFTHVLVPVWIIHYYMKYHVSEKPYGIVEKKSRIFPGDTILETGEVIPPM
KEFPDQHH
```

In [43]:
import pandas as pd
import re 
import sys
import os
import csv

In [2]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/db_UKBccds_set/COUNTING_pipeline/")
print(os.listdir())

['countsfromfastaCCDS_summary_filt_counts.csv', 'Screen Shot 2019-04-25 at 12.30.44 PM.png', 'sumsAA_CCDS_sumsAA_common.csv', 'countsfromfastaCCDS_filter_counts_all_AA.csv', 'CYSpositionsCCDS.csv', 'countsfromfastaCCDS_summary_filt_normalize.csv', 'sumsAA_CCDS_sumsAA.csv', 'countsfromfastaCCDS_filter_normalize_all_AA.csv', 'countsfromfastaCCDSfailQC_data.csv']


In [3]:
ukb = pd.read_csv("countsfromfastaCCDS_filter_counts_all_AA.csv")

In [4]:
ukb.columns

Index(['Unnamed: 0', 'ID', 'entryName', 'Length', 'proSequence', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
      dtype='object')

In [46]:
keepme = ['ID', 'proSequence']
uniprot = ukb[keepme]


In [8]:
# how many duplicates are there
duplicates = uniprot[uniprot.duplicated(['proSequence'],keep=False)]
duplicates = duplicates.sort_values(['proSequence'],ascending=True)

In [9]:
print(uniprot.shape)
print(duplicates.shape)

(18432, 2)
(56, 2)


In [18]:
uniquedups = duplicates['proSequence'].unique()
len(uniquedups)

24

In [21]:
duplicates.head(5) # looks like 24 will be merged
# there should be final size 18400 after merge of duplicated rows

,ID,proSequence
883,P86791,MAAAAAGAGSGPWAAQEKQFPPALLSFFIYNPRFGPREGQEENKIL...
12020,P86790,MAAAAAGAGSGPWAAQEKQFPPALLSFFIYNPRFGPREGQEENKIL...
10521,P0DP23,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
10243,P0DP24,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...
10244,P0DP25,MADQLTEEQIAEFKEAFSLFDKDGDGTITTKELGTVMRSLGQNPTE...


In [22]:
# combine ID and entryName col
%time d1 = {k: g["ID"].tolist() for k,g in uniprot.groupby("proSequence")}

CPU times: user 2.25 s, sys: 29.5 ms, total: 2.28 s
Wall time: 2.26 s


In [30]:
len(d1) # perfect

18400

In [28]:
# turn list of ID's into a string joined by "|"
for key, value in d1.items():
    d1.update({key: "|".join(value)})

In [32]:
merged = pd.DataFrame.from_dict(d1,orient='index')
print(merged.columns)

print(merged.isna().sum())

RangeIndex(start=0, stop=1, step=1)
RangeIndex(start=0, stop=1, step=1)
0    0
dtype: int64


In [37]:
merged.reset_index(inplace=True)
print(merged.columns)

Index(['level_0', 'index', 'proSequence', 'ID_s'], dtype='object')


In [39]:
merged = merged[['proSequence', 'ID_s']].copy()
merged.head(3)

,proSequence,ID_s
0,MAAAAAAAAAAGAAGGRGSGPGRRRHLVPGAGGEAGEGAPGGAGDY...,Q86U42
1,MAAAAAAAAATNGTGGSSGMEVDAAVVPSVMACGVTGSVSVALHPL...,Q7L5N1
2,MAAAAAAAAAVGVRLRDCCSRGAVLLLFFSLSPRPPAAAAWLLGLR...,Q9NRU3


In [41]:
m = merged[['ID_s','proSequence']]
m.head(3)

,ID_s,proSequence
0,Q86U42,MAAAAAAAAAAGAAGGRGSGPGRRRHLVPGAGGEAGEGAPGGAGDY...
1,Q7L5N1,MAAAAAAAAATNGTGGSSGMEVDAAVVPSVMACGVTGSVSVALHPL...
2,Q9NRU3,MAAAAAAAAAVGVRLRDCCSRGAVLLLFFSLSPRPPAAAAWLLGLR...


In [42]:
m.to_csv("UKBccds_NONredundantSequence_18400.csv", index=False)

In [45]:
# put into fasta format
with open("UKBccds_NONredundantSequence_18400.csv",'r') as infile:
    csvReader = csv.reader(infile)
    header = next(csvReader)
    # on data line now

    out = open("UKBcccds_NONredundantSequence.fasta",'w+')

    for row in csvReader:
        ID = str(row[0])
        seq = str(row[1])
        line = ">sp|" + ID + "\n" + seq + "\n"
        out.write(line)
    
out.close()